In [ ]:
# Do not click on "Save Version"
!pip install efficientnet

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


import os
import numpy as np
import cv2
import pickle
import random

import matplotlib.pyplot as plt

import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.models import Sequential, Model, Input, model_from_json
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, concatenate, Lambda, BatchNormalization,AveragePooling2D
from keras import metrics

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder


import efficientnet as efn
import tensorflow.keras
import numpy as np
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from tensorflow.python.keras.layers import Dense,Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, GlobalAveragePooling2D


from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input,decode_predictions
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import efficientnet.keras as efn

In [ ]:
path = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/'

p_path = '../input/chest-xray-pneumonia/chest_xray/train/'
#n_path= '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'
#classes = ['Normal', 'COVID', 'Lung_Opacity', 'Viral Pneumonia']
#classes = ['Normal', 'COVID', 'Viral Pneumonia']
classes0 = ['Normal']
classes1 = ['COVID']
classes2 = ['PNEUMONIA']
X = []
y = []
size = (64, 64)

for cat in range(len(classes0)):
    #images = []
    
    normal_path = os.path.join(path, classes0[cat])
    print(normal_path, ':', len(os.listdir(normal_path)))
    for image_name in random.sample(os.listdir(normal_path), 10):
        try:
            img = cv2.imread(os.path.join(normal_path, image_name))
            img = cv2.resize(img, size)
            img = np.array(img)
            img = np.reshape(img, (64, 64, 3))
            img = img / 255
            #images.append(img)
            X.append(img)
            y.append(0)
        except:
            print(os.path.join(normal_path, image_name))
    #X.append(images)

for cat in range(len(classes1)):
    #images = []
    full_path = os.path.join(path, classes1[cat])
    print(full_path, ':', len(os.listdir(full_path)))
    for image_name in random.sample(os.listdir(full_path), 10):
        try:
            img = cv2.imread(os.path.join(full_path, image_name))
            img = cv2.resize(img, size)
            img = np.array(img)
            img = np.reshape(img, (64, 64, 3))
            img = img / 255
            #images.append(img)
            X.append(img)
            y.append(1)
        except:
            print(os.path.join(full_path, image_name))
    #X.append(images)

    
for cat in range(len(classes2)):
    #images = []
    pneumonia_path = os.path.join(p_path, classes2[cat])
    print(pneumonia_path, ':', len(os.listdir(pneumonia_path)))
    for image_name in random.sample(os.listdir(pneumonia_path), 10):
        try:
            img = cv2.imread(os.path.join(pneumonia_path, image_name))
            img = cv2.resize(img, size)
            img = np.array(img)
            img = np.reshape(img, (64, 64, 3))
            img = img / 255
            #images.append(img)
            X.append(img)
            y.append(2)
        except:
            print(os.path.join(pneumonia_path, image_name))
    #X.append(images)

In [ ]:
def make_pairs(x, y, p=1):
    num_classes = 3
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    pairs = []
    labels = []

    for idx1 in range(len(x)):
        for _ in range(p):
            # add a matching example
            x1 = x[idx1]
            #print(y[idx1])
            label1 = y[idx1]
            #print(label1[0].shape, type(label1[0][0]))
            #print(digit_indices[label1])
            idx2 = random.choice(digit_indices[label1])
            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [1]

            # add a non-matching example
            label2 = random.randint(0, num_classes - 1)
            while label2 == label1:
                label2 = random.randint(0, num_classes - 1)

            idx2 = random.choice(digit_indices[label2])
            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")


# pairs_train, labels_train = make_pairs(x_train, y_train)
# pairs_test, labels_test = make_pairs(x_test, y_test)

In [ ]:
def make_pairs(x, y):
    pairs = [[], []]
    labels = []
    
    for idx1 in range(len(x)):
        for idx2 in range(len(x)):
            pairs[0].append(x[idx1])
            pairs[1].append(x[idx2])
            labels.append(y[idx1] == y[idx2])
    
    return np.array(pairs), np.array(labels).astype("float32")

pairs_train, labels_train = make_pairs(X, y)

In [ ]:
def change_model(model, new_input_shape=(None, 64, 64, 3),custom_objects=None):
    # replace input shape of first layer
    
    config = model.layers[0].get_config()
    config['batch_input_shape']=new_input_shape
    model._layers[0]=model.layers[0].from_config(config)

    # rebuild model architecture by exporting and importing via json
    new_model = tensorflow.keras.models.model_from_json(model.to_json(),custom_objects=custom_objects)

    # copy weights from old model to new one
    for layer in new_model._layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
            print("Loaded layer {}".format(layer.name))
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    return new_model

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

      Arguments:
          y_true: List of labels, each label is of type float32.
          y_pred: List of predictions of same length as of y_true,
                  each label is of type float32.

      Returns:
          A tensor containing constrastive loss as floating point value.
      """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [ ]:
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(64,64,3))
new_model = change_model(base_model, new_input_shape=[None] + [64,64,3])
new_model.summary()
x = new_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
embeddings = Dense(128, activation='relu')(x)
# predictions = Dense(3, activation='sigmoid')(x)

# embedding_network = keras.Model(new_model.input, outputs=predictions)
embedding_network = keras.Model(new_model.input, outputs=embeddings)

input_1 = Input((64, 64, 3))
input_2 = Input((64, 64, 3))

tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [ ]:
epochs = 10
batch_size = 3
margin = 1  # Margin for constrastive loss.

In [ ]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

In [ ]:
pairs_train.shape, labels_train.shape

In [ ]:
history = siamese.fit((pairs_train[0], pairs_train[1]), labels_train, epochs=epochs, batch_size=batch_size)

In [ ]:
plot_model(
    siamese, to_file='siamese_nimai_model.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=True, dpi=96
)

In [ ]:
# load json and create model
json_file = open('../input/efficientnet/best_siamese_efficientB0_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../input/efficientnet/best_siamese_efficientB0.h5")

In [ ]:
# load json and create model
json_file = open('../input/efficientnet/vgg16_siamese_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../input/efficientnet/vgg16_constrastive_loss.h5")

In [ ]:
plot_model(
    loaded_model, to_file='loaded_siamese_nimai_model.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=True, dpi=96
)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape, y.shape

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
y_train = y_train.reshape((y_train.shape[0], 1))
y_test = y_test.reshape((y_test.shape[0], 1))
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

'              precision    recall  f1-score   support
'       0.0       0.96      0.98      0.97      2199
'       1.0       0.98      0.96      0.97      2199
'accuracy                             0.97      4398
'macro avg        0.97      0.97      0.97      4398
'weighted avg     0.97      0.97      0.97      4398

In [ ]:
digit_indices = [np.where(y_train == i)[0] for i in range(3)]
len(digit_indices), len(digit_indices[0]), len(digit_indices[1]), len(digit_indices[2])

In [ ]:
import pandas as pd

classes = [
    'Atelectasis',
    'Cardiomegaly',
    'Consolidation',
    'Edema',
    'Effusion',
    'Emphysema',
    'Fibrosis',
    'Hernia',
    'Infiltration',
    'Pleural_Thickening',
    'Pneumothorax',
    'Pneumonia',
    'Nodule',
    'Mass',
    'No Finding'
]

meta_data = pd.read_csv('/kaggle/input/processeddataforlungdiseases/preprocessed_data.csv')
for i, d in enumerate(classes):
    print(i, d, len(meta_data[meta_data[d] == 1]), len(meta_data[meta_data[d] == 0]))

In [ ]:
meta_data.columns
meta_data['Unnamed: 0'][:5]

In [ ]:
y = [[]] * len(classes)
size = (64, 64)
dataset_len = 5000
test_size = 200

for i in range(len(classes)):
    y[i] = list(meta_data[:dataset_len][classes[i]])
y = np.array(y).T

In [ ]:
y.shape

In [ ]:
from tqdm.notebook import tqdm
X = []
for path in tqdm(meta_data[:dataset_len]['Unnamed: 0'], total=dataset_len):
    try:
        img = cv2.imread(path)
        img = cv2.resize(img, size)
        img = np.array(img)
        img = np.reshape(img, (64, 64, 3))
        img = img / 255

        X.append(img)
    except:
        print(os.path.join(normal_path, image_name))
X = np.array(X)

In [ ]:
shots = 8
preds = []
accuracies = []

support_set_indices_positive = []
support_set_indices_negative = []
support_set_positive = []
support_set_negative = []

for cat in range(len(classes)):
    support_set_indices_positive.append(random.sample(list(np.where(y[:, cat] == 1)[0]), shots))
    support_set_positive.append(list(X[support_set_indices_positive[-1]]))
    support_set_indices_negative.append(random.sample(list(np.where(y[:, cat] == 0)[0]), shots))
    support_set_negative.append(list(X[support_set_indices_negative[-1]]))
support_set_positive = np.array(support_set_positive)
support_set_negative = np.array(support_set_negative)

In [ ]:
preds = []
test_set = random.sample(range(len(X)), test_size)
correct_pred = [0] * len(classes)

for i in tqdm(test_set, total=test_size):
    temp = np.repeat([X[i]], shots, axis=0)
    pred = []
    for cat in range(len(classes)):
        pred_positive = np.sum(np.array(loaded_model.predict([support_set_positive[cat], temp])))
        pred_negative = np.sum(np.array(loaded_model.predict([support_set_negative[cat], temp])))
        pred_temp = int(pred_positive > pred_negative)
        pred.append(pred_temp)
        correct_pred[cat] += pred_temp == y[i][cat]
    preds.append([pred])
#     print(pred, y[i])
preds = np.array(preds).T
# print(preds)
print(np.array(correct_pred)/test_size)

In [ ]:
for r in zip(classes, np.array(correct_pred)/test_size):
    print(r)

In [ ]:
test_size = len(X)
shots = 8
preds = []
accuracies = []

for shot in range(1, shots+1):
    normal = random.sample(list(digit_indices[0]), shot)
    covid = random.sample(list(digit_indices[1]), shot)
    pneumonia = random.sample(list(digit_indices[2]), shot)
    
    normal_imgs = x_train[normal]
    covid_imgs = x_train[covid]
    pneumonia_imgs = x_train[pneumonia]
    
    temp_pred = []
    
    for test_img in tqdm(x_test[:test_size], total=test_size):
        temp = np.repeat([test_img], shot, axis=0)
        # print(normal_imgs.shape, temp.shape)
        pred_0 = np.array(loaded_model.predict([normal_imgs, temp]) > 0.5)
        # print(pred_0)
        pred_0 = np.sum(pred_0)
        # print(pred_0)
        
        pred_1 = np.array(loaded_model.predict([covid_imgs, temp]) > 0.5)
        # print(pred_1)
        pred_1 = np.sum(pred_1)
        # print(pred_1)
        
        pred_2 = np.array(loaded_model.predict([pneumonia_imgs, temp]) > 0.5)
        # print(pred_2)
        pred_2 = np.sum(pred_2)
        # print(pred_2)
        
        # print(np.argmax([pred_0, pred_1, pred_2]))
        temp_pred.append(np.argmax([pred_0, pred_1, pred_2]))
        
        # print()
    # print('preds', preds)
    # print('y_test', y_test[:test_size])
    print(shot, 'shot accuracy =', (np.sum(temp_pred == y_test[:test_size]) * 100) / test_size)
    preds.append(temp_pred)
    accuracies.append((np.sum(temp_pred == y_test[:test_size]) * 100) / test_size)
    print()

In [ ]:
for p in preds:
    print()
    print(confusion_matrix(y_test, p))
    print()
    print(classification_report(y_test, p))
    print()
    
# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199



# [[685  20   7]
#  [ 13 717   2]
#  [  1   3 751]]

#               precision    recall  f1-score   support

#            0       0.98      0.96      0.97       712
#            1       0.97      0.98      0.97       732
#            2       0.99      0.99      0.99       755

#     accuracy                           0.98      2199
#    macro avg       0.98      0.98      0.98      2199
# weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       712
           1       0.97      0.98      0.97       732
           2       0.99      0.99      0.99       755

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199

In [ ]:
accuracies
# [97.9081400636653,
#  97.9081400636653,
#  97.9081400636653,
#  97.9081400636653,
#  97.9081400636653,
#  97.9081400636653,
#  97.9081400636653,
#  97.9081400636653,
#  97.9081400636653,
#  97.9081400636653]

In [ ]:
test_size = len(x_test)
shot = 8

normal = random.sample(list(digit_indices[0]), shot)
covid = random.sample(list(digit_indices[1]), shot)
pneumonia = random.sample(list(digit_indices[2]), shot)

normal_imgs = x_train[normal]
covid_imgs = x_train[covid]
pneumonia_imgs = x_train[pneumonia]

pred_vals = []

for test_img in tqdm(x_test[:test_size], total=test_size):
    temp = np.repeat([test_img], shot, axis=0)
    pred_0 = np.array(loaded_model.predict([normal_imgs, temp]))
#     print(pred_0)

    pred_1 = np.array(loaded_model.predict([covid_imgs, temp]))
#     print(pred_1)

    pred_2 = np.array(loaded_model.predict([pneumonia_imgs, temp]))
#     print(pred_2)
    
    pred_vals.append(np.array([pred_0, pred_1, pred_2]))
#     print()

pred_vals = np.array(pred_vals)

In [ ]:
print(pred_vals.shape)
sum1 = np.sum(pred_vals[:4], axis=2)/8
print(sum1.shape)
ohe = OneHotEncoder()
temp = ohe.fit_transform(y_test).toarray()
# for i in range(4):
#     print(temp[i], sum1[i])
print(temp.shape)
temp = temp.reshape((temp.shape[0], temp.shape[1], 1))
print(temp.shape)

In [ ]:
# normal vs not normal
tpr = []
fpr = []
step = 0.00005
for threshold in tqdm(np.arange(0, 1+step, step), total=len(np.arange(0, 1+step, step))):
    preds = []
    temp = pred_vals[:] >= threshold
    # print(temp)
    temp = np.sum(temp, axis=2)
    # print(temp)
    # print(np.argmax(temp, axis=1))
    preds = np.argmax(temp, axis=1)
    # print(preds)
    preds_normal = preds != 0
    # print(preds)

    cm = confusion_matrix(y_test != 0, preds_normal)
    tpr.append(cm[0][0] / (cm[0][0] + cm[0][1]))
    fpr.append(cm[1][0] / (cm[1][0] + cm[1][1]))

In [ ]:
confusion_matrix(y_test != 0, preds_normal)

In [ ]:
plt.figure(figsize=(20,10))
plt.grid()
plt.plot(fpr, tpr, 'ro--')
plt.plot([0,1], [0,1], '--')
plt.show()

In [ ]:
predictions = pred_vals[:] >= 0.9999
predictions = np.sum(predictions, axis=2)
predictions = np.argmax(predictions, axis=1)

In [ ]:
predictions = predictions.reshape(predictions.shape[0])
predictions.shape, y_test.shape

In [ ]:
confusion_matrix(y, predictions), np.sum(y == predictions) * 100 / len(y)

In [ ]:
digit_indices_support = [np.where(y_train == i)[0] for i in range(3)]
print(len(digit_indices_support), len(digit_indices_support[0]), len(digit_indices_support[1]), len(digit_indices_support[2]))

digit_indices_test = [np.where(y_test == i)[0] for i in range(3)]
print(len(digit_indices_test), len(digit_indices_test[0]), len(digit_indices_test[1]), len(digit_indices_test[2]))

In [ ]:
size = len(x_test)
shot = 8

normal = random.sample(list(digit_indices_support[0]), shot)
covid = random.sample(list(digit_indices_support[1]), shot)
pneumonia = random.sample(list(digit_indices_support[2]), shot)

normal_imgs = x_train[normal]
covid_imgs = x_train[covid]
pneumonia_imgs = x_train[pneumonia]

pred_vals = []

for test_img in tqdm(x_test[np.concatenate((digit_indices_test[1], digit_indices_test[2]))][:], total=len(np.concatenate((digit_indices_test[1], digit_indices_test[2])))):
    temp = np.repeat([test_img], shot, axis=0)
    
    # pred_0 = np.array(loaded_model.predict([normal_imgs, temp]))
    # print(pred_0)
    # print(pred_0.shape)
    # pred_0 = np.sum(pred_0) / shot
    # print(pred_0)
    # print(pred_0.shape)
    # pred_vals.append(pred_0)

    # pred_1 = np.array(loaded_model.predict([covid_imgs, temp]))
    # print(pred_1)
    # print(pred_1.shape)
    # pred_1 = np.sum(pred_1) / shot
    # print(pred_1)
    # print(pred_1.shape)
    # pred_vals.append(pred_1)

    pred_2 = np.array(loaded_model.predict([pneumonia_imgs, temp]))
    # print(pred_2)
    # print(pred_2.shape)
    pred_2 = np.sum(pred_2) / shot
    # print(pred_2)
    # print(pred_2.shape)
    pred_vals.append(pred_2)
    
    # print()

pred_vals = np.array(pred_vals)
pred_vals.shape

In [ ]:
print(y_test[:20])
print(y_test[np.concatenate((digit_indices_test[1], digit_indices_test[2]))][:20])

In [ ]:
def make_pairs_test(x_train, y_train, x_test, y_test, p=1):
    num_classes = 3
    digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]

    pairs = []
    labels = []

    for idx1 in range(len(x_test)):
        for _ in range(p):
            # add a matching example
            x1 = x_test[idx1]
            #print(y[idx1])
            label1 = y_test[idx1]
            #print(label1[0].shape, type(label1[0][0]))
            #print(digit_indices[label1])
            idx2 = random.choice(digit_indices[label1])
            x2 = x_train[idx2]

            pairs += [[x1, x2]]
            labels += [1]

            # add a non-matching example
            label2 = random.randint(0, num_classes - 1)
            while label2 == label1:
                label2 = random.randint(0, num_classes - 1)

            idx2 = random.choice(digit_indices[label2])
            x2 = x_train[idx2]

            pairs += [[x1, x2]]
            labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")

In [ ]:
X, y = make_pairs_test(x_train, y_train, x_test, y_test)

x1 = X[:, 0]
x2 = X[:, 1]

In [ ]:
x1.shape, x2.shape, y. shape

In [ ]:
from sklearn.metrics import roc_curve, auc
#  = np.array(y_test[np.concatenate((digit_indices_test[1], digit_indices_test[2]))] == 1, dtype='int')
fpr, tpr, thresholds_keras = roc_curve(y_test[np.concatenate((digit_indices_test[1], digit_indices_test[2]))] == 2, pred_vals)
auc_val = auc(fpr, tpr)

In [ ]:
from sklearn.metrics import roc_curve, auc
y_pred = loaded_model.predict([x1, x2])
fpr, tpr, thresholds_keras = roc_curve(y, y_pred)
auc_val = auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(20, 10))
plt.grid()
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='area = {:.3f}'.format(auc_val))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend(loc='lower right')
plt.show()

In [ ]:
print(classification_report(y, y_pred > 0.5))

In [ ]:
# load json and create model
json_file = open('../input/efficientnet/best_embedding_efficientB0_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../input/efficientnet/best_embedding_efficientB0.h5")

In [ ]:
plot_model(
    loaded_model, to_file='loaded_siamese_nimai_model.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=True, dpi=96
)